In [1]:
import pickle
import numpy as np
import pandas as pd
import copy
import os
import sys
sys.path.append("../")
from Schemas.imdb.schema import gen_imdb_schema
from Join_scheme.data_prepare import read_table_csv, process_stats_data, process_imdb_data
from Join_scheme.bound import Bound_ensemble
from Join_scheme.join_graph import parse_query_all_join, get_join_hyper_graph
from Evaluation.testing import get_job_sub_plan_queires

In [2]:
n_bins = {
            'title.id': 800,
            'info_type.id': 100,
            'keyword.id': 100,
            'company_name.id': 100,
            'name.id': 100,
            'company_type.id': 100,
            'comp_cast_type.id': 50,
            'kind_type.id': 50,
            'char_name.id': 50,
            'role_type.id': 50,
            'link_type.id': 50
        }
data_path = "/Users/ziniuw/Desktop/research/Learned_QO/data/imdb/{}.csv"
model_folder = "/Users/ziniuw/Desktop/research/Learned_QO/data/saved_models"
schema, table_buckets, ground_truth_factors_no_filter = process_imdb_data(data_path, model_folder, n_bins, "fixed_start_key",
                                                                              save_bucket_bins=True)

/Users/ziniuw/opt/anaconda3/envs/bayescard/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (5,10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/ziniuw/opt/anaconda3/envs/bayescard/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/ziniuw/opt/anaconda3/envs/bayescard/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/ziniuw/opt/anaconda3/envs/bayescard/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=Fal

title.kind_id
movie_info.info_type_id
movie_keyword.movie_id
cast_info.person_id
cast_info.person_role_id
cast_info.role_id
complete_cast.status_id
movie_keyword.keyword_id
movie_companies.company_id
movie_companies.company_type_id
kind_type.id 8
info_type.id 72
title.id 74
name.id 27
char_name.id 28
role_type.id 12
comp_cast_type.id 2
keyword.id 86
company_name.id 75
company_type.id 3


In [16]:
import time
import glob
import os
import pickle
from multiprocessing import Pool
import networkx as nx
from networkx.readwrite import json_graph
from Sampling.utils.query_storage import load_sql_rep, save_sql_rep
from Sampling.utils.parse_sql import parse_sql
import collections.abc
import sqlparse
#hyper needs the four following aliases to be done manually.
collections.Iterable = collections.abc.Iterable
collections.Mapping = collections.abc.Mapping
collections.MutableSet = collections.abc.MutableSet
collections.MutableMapping = collections.abc.MutableMapping

fn = "/Users/ziniuw/Desktop/research/Learned_QO/factorjoin-binned-cards/queries/job/all_job/17a.pkl"
qrep = load_sql_rep(fn)

In [11]:
node_list = list(qrep["subset_graph"].nodes())
node_list.sort(reverse=True, key=lambda x: len(x))
print(node_list)

[('ci', 'cn', 'k', 'mc', 'mk', 'n', 't'), ('ci', 'cn', 'k', 'mc', 'mk', 't'), ('ci', 'cn', 'mc', 'mk', 'n', 't'), ('ci', 'k', 'mc', 'mk', 'n', 't'), ('ci', 'cn', 'k', 'mc', 'mk', 'n'), ('ci', 'k', 'mc', 'mk', 't'), ('cn', 'k', 'mc', 'mk', 't'), ('ci', 'k', 'mk', 'n', 't'), ('ci', 'cn', 'mc', 'n', 't'), ('ci', 'cn', 'mc', 'mk', 'n'), ('ci', 'cn', 'k', 'mc', 'mk'), ('ci', 'k', 'mc', 'mk', 'n'), ('ci', 'cn', 'mc', 'mk', 't'), ('ci', 'mc', 'mk', 'n', 't'), ('k', 'mc', 'mk', 't'), ('ci', 'cn', 'mc', 'n'), ('ci', 'mc', 'n', 't'), ('ci', 'mc', 'mk', 'n'), ('ci', 'cn', 'mc', 't'), ('ci', 'k', 'mk', 'n'), ('cn', 'k', 'mc', 'mk'), ('ci', 'k', 'mk', 't'), ('ci', 'mc', 'mk', 't'), ('ci', 'mk', 'n', 't'), ('ci', 'cn', 'mc', 'mk'), ('ci', 'k', 'mc', 'mk'), ('cn', 'mc', 'mk', 't'), ('cn', 'mc', 't'), ('mc', 'mk', 't'), ('ci', 'mc', 'n'), ('cn', 'mc', 'mk'), ('ci', 'n', 't'), ('k', 'mk', 't'), ('ci', 'cn', 'mc'), ('ci', 'k', 'mk'), ('k', 'mc', 'mk'), ('ci', 'mk', 't'), ('ci', 'mc', 't'), ('ci', 'mk', 

In [15]:
sg = qrep["join_graph"].subgraph(('mk',))
tname = qrep["join_graph"].nodes()['mk']["real_name"]
print(sg, tname)

 movie_keyword


In [33]:
parsed = sqlparse.parse(qrep['sql'])[0]
from_clause = None
from_seen = False
where_clauses = None
for token in parsed.tokens:
    if isinstance(token, sqlparse.sql.Where):
        where_clauses = token
for token in parsed.tokens:
    if from_seen:
        if isinstance(token, sqlparse.sql.IdentifierList) or isinstance(token,
                sqlparse.sql.Identifier):
            from_clause = token
            break
    if token.value.upper() == 'FROM':
        from_seen = True



In [34]:
print(from_clause)

cast_info AS ci,
     company_name AS cn,
     keyword AS k,
     movie_companies AS mc,
     movie_keyword AS mk,
     name AS n,
     title AS t


In [46]:
import sqlparse


def find_next_match(tables, wheres, index):
    '''
    ignore everything till next
    '''
    match = ""
    _, token = wheres.token_next(index)
    if token is None:
        return None, None
    # FIXME: is this right?
    if token.is_keyword:
        index, token = wheres.token_next(index)

    tables_in_pred = find_all_tables_till_keyword(token)
    print(tables_in_pred)
    assert len(tables_in_pred) <= 2

    token_list = sqlparse.sql.TokenList(wheres)

    while True:
        index, token = token_list.token_next(index)
        if token is None:
            break
        # print("token.value: ", token.value)
        if token.value.upper() == "AND":
            break

        match += " " + token.value

        if (token.value.upper() == "BETWEEN"):
            # ugh ugliness
            index, a = token_list.token_next(index)
            index, AND = token_list.token_next(index)
            index, b = token_list.token_next(index)
            match += " " + a.value
            match += " " + AND.value
            match += " " + b.value
            # Note: important not to break here! Will break when we hit the
            # "AND" in the next iteration.
    print(match)
    # print("tables: ", tables)
    # print("match: ", match)
    # print("tables in pred: ", tables_in_pred)
    for table in tables_in_pred:
        if table not in tables:
            # print(tables)
            # print(table)
            # pdb.set_trace()
            # print("returning index, None")
            return index, None

    if len(tables_in_pred) == 0:
        return index, None

    return index, match


def find_all_tables_till_keyword(token):
    tables = []
    # print("fattk: ", token)
    index = 0
    while (True):
        if (type(token) == sqlparse.sql.Comparison):
            left = token.left
            right = token.right
            if (type(left) == sqlparse.sql.Identifier):
                tables.append(left.get_parent_name())
            if (type(right) == sqlparse.sql.Identifier):
                tables.append(right.get_parent_name())
            break
        elif (type(token) == sqlparse.sql.Identifier):
            tables.append(token.get_parent_name())
            break
        try:
            index, token = token.token_next(index)
            if ("Literal" in str(token.ttype)) or token.is_keyword:
                break
        except:
            break

    return tables


def find_filter_clauses(table, wheres):
    matched = []
    # print(tables)
    index = 0
    while True:
        index, match = find_next_match(table, wheres, index)
        # print("got index, match: ", index)
        # print(match)
        if match is not None:
            matched.append(match)
        if index is None:
            break

    return matched

In [48]:
find_filter_clauses(['cn'], where_clauses)

['cn']
 cn.country_code ='[us]'
['k']
 k.keyword ='character-name-in-title'
['n']
 n.name LIKE 'B%'
['n', 'ci']
 n.id = ci.person_id
['ci', 't']
 ci.movie_id = t.id
['t', 'mk']
 t.id = mk.movie_id
['mk', 'k']
 mk.keyword_id = k.id
['t', 'mc']
 t.id = mc.movie_id
['mc', 'cn']
 mc.company_id = cn.id
['ci', 'mc']
 ci.movie_id = mc.movie_id
['ci', 'mk']
 ci.movie_id = mk.movie_id
['mc', 'mk']
 mc.movie_id = mk.movie_id ;


[" cn.country_code ='[us]'"]

In [ ]:
be = Bound_ensemble(table_buckets, schema, 1, ground_truth_factors_no_filter)

In [ ]:
model_path = model_folder + f"model_imdb_default.pkl"
pickle.dump(be, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)

In [3]:
with open("/Users/ziniuw/Desktop/research/Learned_QO/FactorJoin/factorjoin-binned-cards/bins.pkl", "rb") as f:
    prev_bins = pickle.load(f)

In [4]:
with open("/Users/ziniuw/Desktop/research/Learned_QO/data/saved_models/imdb_buckets.pkl", "rb") as f:
    curr_bins = pickle.load(f)

In [5]:
attr = "title.id"
print(prev_bins.keys())
for attr in list(prev_bins.keys()):
    print(attr, "-----------------------------------------------")
    print(len(prev_bins[attr]), len(prev_bins[attr][0]))
    print(len(curr_bins[attr].bins), len(curr_bins[attr].bins[0]))

dict_keys(['kind_type.id', 'info_type.id', 'title.id', 'name.id', 'char_name.id', 'role_type.id', 'comp_cast_type.id', 'keyword.id', 'company_name.id', 'company_type.id'])
kind_type.id -----------------------------------------------
8 1
8 1
info_type.id -----------------------------------------------
72 3
72 3
title.id -----------------------------------------------
74 1238985
74 1224796
name.id -----------------------------------------------
29 781509
27 806247
char_name.id -----------------------------------------------
28 269322
28 269322
role_type.id -----------------------------------------------
12 1
12 1
comp_cast_type.id -----------------------------------------------
2 2
2 1
keyword.id -----------------------------------------------
86 31359
86 31359
company_name.id -----------------------------------------------
75 80873
75 80873
company_type.id -----------------------------------------------
3 1
3 1


In [ ]:
import copy
import logging
import pickle
import numpy as np
import pandas as pd
import time
import os

from Schemas.stats.schema import gen_stats_light_schema
from Schemas.imdb.schema import gen_imdb_schema
from Join_scheme.binning import identify_key_values, sub_optimal_bucketize, greedy_bucketize, \
                                fixed_start_key_bucketize, get_start_key, naive_bucketize, \
                                Table_bucket, update_bins, apply_binning_to_data_value_count
from Join_scheme.bound import Factor


data_path = "/Users/ziniuw/Desktop/research/Learned_QO/data/imdb/{}.csv"
model_folder = "/Users/ziniuw/Desktop/research/Learned_QO/data/saved_models"
bucket_method = "fixed_start_key"
sample_size=1000000
save_bucket_bins=False
seed=0

schema = gen_imdb_schema(data_path)
all_keys, equivalent_keys = identify_key_values(schema)
data = dict()
table_lens = dict()
table_key_lens = dict()
na_values = dict()
primary_keys = []
for table_obj in schema.tables:
    df_rows = pd.read_csv(table_obj.csv_file_location, header=None, escapechar='\\', encoding='utf-8',
                          quotechar='"',
                          sep=",")

    df_rows.columns = [table_obj.table_name + '.' + attr for attr in table_obj.attributes]
    for attribute in table_obj.irrelevant_attributes:
        df_rows = df_rows.drop(table_obj.table_name + '.' + attribute, axis=1)

    df_rows.apply(pd.to_numeric, errors="ignore")
    table_lens[table_obj.table_name] = len(df_rows)
    if table_obj.table_name not in na_values:
        na_values[table_obj.table_name] = dict()
    for attr in df_rows.columns:
        if attr in all_keys:
            table_key_lens[attr] = table_lens[table_obj.table_name]
            data[attr] = df_rows[attr].values
            data[attr][np.isnan(data[attr])] = -1
            data[attr][data[attr] < 0] = -1
            na_values[table_obj.table_name][attr] = len(data[attr][data[attr] != -1]) / table_lens[
                table_obj.table_name]
            data[attr] = copy.deepcopy(data[attr])[data[attr] >= 0]
            if len(np.unique(data[attr])) >= len(data[attr]) - 10:
                primary_keys.append(attr)


In [ ]:
value_count_var = dict()
for attr in data:
    #print(attr)
    u, v = np.unique(data[attr], return_counts=True)
    #print(len(u), len(v), np.var(v))
    value_count_var[attr] = np.var(v)

In [ ]:
data.keys()

In [ ]:
for PK in equivalent_keys:
    start_key = get_start_key(equivalent_keys[PK], value_count_var, primary_keys)
    print(start_key)

In [ ]:
prev_bins["role_type.id"]

In [ ]:
curr_bins["role_type.id"].bins

In [ ]:
len(prev_bins['title.id'])

In [ ]:
len(curr_bins['title.id'].bins)

In [ ]:
model_folder = "/Users/ziniuw/Desktop/research/Learned_QO/data/saved_models"
model_path = model_folder + f"model_imdb_default.pkl"
with open(model_path, "rb") as f:
    be = pickle.load(f)

In [ ]:
be.SPERCENTAGE = 1.0
be.query_sample_location = "../Sampling/materialized_IMDB_JOB_sample/{}/job/all_job/"
query_folder = "../IMDB-JOB/"

In [ ]:
all_queries, all_sub_plan_queries_str = get_job_sub_plan_queires(query_folder)

In [ ]:
q_name = "19b"
temp = be.get_cardinality_bound_all(all_queries[q_name], all_sub_plan_queries_str[q_name], q_name + ".pkl")

In [ ]:
data_path = "/home/ubuntu/End-to-End-CardEst-Benchmark/datasets/stats_simplified/{}.csv"
schema = gen_stats_light_schema(data_path)

In [ ]:
all_keys, equivalent_keys = identify_key_values(schema)
print(equivalent_keys)

In [ ]:
data = dict()
sample_rate = dict()
primary_keys = []
for table_obj in schema.tables:
    df_rows = pd.read_csv(table_obj.csv_file_location)
    
    df_rows.columns = [table_obj.table_name + '.' + attr for attr in table_obj.attributes]

    for attribute in table_obj.irrelevant_attributes:
        df_rows = df_rows.drop(table_obj.table_name + '.' + attribute, axis=1)

    df_rows.apply(pd.to_numeric, errors="ignore")
    for attr in df_rows.columns:
        if attr in all_keys:
            print(attr)
            print(np.sum(np.isnan(df_rows[attr].values)))
            data[attr] = df_rows[attr].values
            data[attr][np.isnan(data[attr])] = -1
            data[attr][data[attr] < 0] = -1
            print(len(data[attr]), len(df_rows[attr].values))
            data[attr] = copy.deepcopy(data[attr])[data[attr]>=0]
            print(len(data[attr]), len(df_rows[attr].values))
            if len(np.unique(data[attr])) >= len(data[attr]) - 10:
                primary_keys.append(attr)
            print(np.sum(np.isnan(df_rows[attr].values)))

In [ ]:
for k in data:
    sample_rate[k] = 1.0
    print(k, len(data[k]), np.sum(np.isnan(data[k])), len(data[k][data[k]<0]))
print(primary_keys)


In [ ]:
temp = dict()
for PK in equivalent_keys:
    print(equivalent_keys[PK])
    group_data = {}
    group_sample_rate = {}
    for K in equivalent_keys[PK]:
        group_data[K] = data[K]
        group_sample_rate[K] = sample_rate[K]
    d, bucket = naive_bucketize(group_data, sample_rate, n_bins=100, primary_keys=primary_keys, return_data=True)
    temp[PK] = bucket
    for key in d:
        print(np.min(d[key]), np.max(d[key]))

In [ ]:
print(len(bucket.buckets[bucket.start_key].bins))
for k in d:
    print("     ==      ")
    print(k, len(data[k]), np.sum(np.isnan(data[k])), len(data[k][data[k]<0]))
    print(len(d[k]), len(np.unique(d[k])))

In [ ]:
bucket = temp["posts.Id"]
for k in bucket.buckets:
    print("==============================================================")
    print(k, len(bucket.buckets[k].bin_modes))
    print(bucket.buckets[k].bin_modes)
    print([len(b) for b in bucket.buckets[k].bins])
    print(bucket.buckets[k].bin_means)
    print(bucket.buckets[k].bin_vars)

In [ ]:
import copy
import logging
import pickle
import numpy as np
import pandas as pd
import time
import sys
import os

sys.path.append("/Users/ziniuw/Desktop/research/Learned_QO/CE_scheme/")

from Schemas.stats.schema import gen_stats_light_schema
from Join_scheme.binning import identify_key_values, sub_optimal_bucketize, greedy_bucketize, Table_bucket

logger = logging.getLogger(__name__)


def timestamp_transorform(time_string, start_date="2010-07-19 00:00:00"):
    start_date_int = time.strptime(start_date, "%Y-%m-%d %H:%M:%S")
    time_array = time.strptime(time_string, "%Y-%m-%d %H:%M:%S")
    return int(time.mktime(time_array)) - int(time.mktime(start_date_int))


def read_table_hdf(table_obj):
    """
    Reads hdf from path, renames columns and drops unnecessary columns
    """
    df_rows = pd.read_hdf(table_obj.csv_file_location)
    df_rows.columns = [table_obj.table_name + '.' + attr for attr in table_obj.attributes]

    for attribute in table_obj.irrelevant_attributes:
        df_rows = df_rows.drop(table_obj.table_name + '.' + attribute, axis=1)

    return df_rows.apply(pd.to_numeric, errors="ignore")


def convert_time_to_int(data_folder):
    for file in os.listdir(data_folder):
        if file.endswith(".csv"):
            csv_file_location = data_folder + file
            df_rows = pd.read_csv(csv_file_location)
            for attribute in df_rows.columns:
                if "Date" in attribute:
                    if df_rows[attribute].values.dtype == 'object':
                        new_value = []
                        for value in df_rows[attribute].values:
                            new_value.append(timestamp_transorform(value))
                        df_rows[attribute] = new_value
            df_rows.to_csv(csv_file_location, index=False)


def read_table_csv(table_obj, csv_seperator=',', stats=True):
    """
    Reads csv from path, renames columns and drops unnecessary columns
    """
    if stats:
        df_rows = pd.read_csv(table_obj.csv_file_location)
    else:
        df_rows = pd.read_csv(table_obj.csv_file_location, header=None, escapechar='\\', encoding='utf-8',
                              quotechar='"',
                              sep=csv_seperator)
    df_rows.columns = [table_obj.table_name + '.' + attr for attr in table_obj.attributes]

    for attribute in table_obj.irrelevant_attributes:
        df_rows = df_rows.drop(table_obj.table_name + '.' + attribute, axis=1)

    return df_rows.apply(pd.to_numeric, errors="ignore")


def generate_table_buckets(data, key_attrs, bin_sizes, bin_modes, optimal_buckets):
    table_buckets = dict()
    for table in data:
        table_data = data[table]
        table_bucket = Table_bucket(table, key_attrs[table], bin_sizes[table])
        for key in key_attrs[table]:
            if key in bin_modes and len(bin_modes[key]) != 0:
                table_bucket.oned_bin_modes[key] = bin_modes[key]
            else:
                # this is a primary key
                table_bucket.oned_bin_modes[key] = np.ones(table_bucket.bin_sizes[key])
        # getting mode for 2D bins
        if len(key_attrs[table]) == 2:
            key1 = key_attrs[table][0]
            key2 = key_attrs[table][1]
            res1 = np.zeros((table_bucket.bin_sizes[key1], table_bucket.bin_sizes[key2]))
            res2 = np.zeros((table_bucket.bin_sizes[key1], table_bucket.bin_sizes[key2]))
            key_data = np.stack((table_data[key1].values, table_data[key2].values), axis=1)
            assert table_bucket.bin_sizes[key1] == len(optimal_buckets[key1].bins)
            assert table_bucket.bin_sizes[key2] == len(optimal_buckets[key2].bins)
            for v1, b1 in enumerate(optimal_buckets[key1].bins):
                temp_data = key_data[np.isin(key_data[:, 0], b1)]
                if len(temp_data) == 0:
                    continue
                #print(key1, v1, np.max(np.unique(temp_data[:, 0], return_counts=True)[-1]), table_bucket.oned_bin_modes[key1][v1])
                #assert np.max(np.unique(temp_data[:, 0], return_counts=True)[-1]) == table_bucket.oned_bin_modes[key1][
                    #v1], f"{key1} data error at {v1}, with " \
                     #    f"{np.max(np.unique(temp_data[:, 0], return_counts=True)[-1])} and " \
                      #   f"{table_bucket.oned_bin_modes[key1][v1]}."
                for v2, b2 in enumerate(optimal_buckets[key2].bins):
                    temp_data2 = copy.deepcopy(temp_data[np.isin(temp_data[:, 1], b2)])
                    if len(temp_data2) == 0:
                        continue
                    res1[v1, v2] = np.max(np.unique(temp_data2[:, 0], return_counts=True)[-1])
                    res2[v1, v2] = np.max(np.unique(temp_data2[:, 1], return_counts=True)[-1])
            table_bucket.twod_bin_modes[key1] = res1
            table_bucket.twod_bin_modes[key2] = res2
        table_buckets[table] = table_bucket

    return table_buckets


def process_stats_data(data_path, model_folder, n_bins=500, bucket_method="greedy", save_bucket_bins=False):
    """
    Preprocessing stats data and generate optimal bucket
    :param data_path: path to stats data folder
    :param n_bins: number of bins (the actually number of bins returned will be smaller than this)
    :param bucket_method: choose between "sub_optimal" and "greedy". Please refer to binning.py for details.
    :param save_bucket_bins: Set to true for dynamic environment, the default is False for static environment
    :return:
    """
    if not data_path.endswith(".csv"):
        data_path += "/{}.csv"
    schema = gen_stats_light_schema(data_path)
    all_keys, equivalent_keys = identify_key_values(schema)
    data = dict()
    key_data = dict()  # store the columns of all keys
    sample_rate = dict()
    primary_keys = []
    null_values = dict()
    key_attrs = dict()
    for table_obj in schema.tables:
        table_name = table_obj.table_name
        null_values[table_name] = dict()
        key_attrs[table_name] = []
        df_rows = read_table_csv(table_obj, stats=True)
        for attr in df_rows.columns:
            if attr in all_keys:
                key_data[attr] = df_rows[attr].values
                # the nan value of id are set to -1, this is hardcoded.
                key_data[attr][np.isnan(key_data[attr])] = -1
                key_data[attr][key_data[attr] < 0] = -1
                null_values[table_name][attr] = -1
                key_data[attr] = copy.deepcopy(key_data[attr])[key_data[attr] >= 0]
                # if the all keys have exactly one appearance, we consider them primary keys
                # we set a error margin of 0.01 in case of data mis-write.
                if len(np.unique(key_data[attr])) >= len(key_data[attr]) * 0.99:
                    primary_keys.append(attr)
                sample_rate[attr] = 1.0
                key_attrs[table_name].append(attr)
            else:
                temp = df_rows[attr].values
                null_values[table_name][attr] = np.nanmin(temp) - 100
                temp[np.isnan(temp)] = null_values[table_name][attr]
        data[table_name] = df_rows

    all_bin_modes = dict()
    bin_size = dict()
    binned_data = dict()
    optimal_buckets = dict()
    for PK in equivalent_keys:
        print(f"bucketizing equivalent key group:", equivalent_keys[PK])
        group_data = {}
        group_sample_rate = {}
        for K in equivalent_keys[PK]:
            group_data[K] = key_data[K]
            group_sample_rate[K] = sample_rate[K]
        if bucket_method == "greedy":
            temp_data, optimal_bucket = greedy_bucketize(group_data, sample_rate, n_bins, primary_keys, True)
        elif bucket_method == "sub_optimal":
            temp_data, optimal_bucket = sub_optimal_bucketize(group_data, sample_rate, n_bins, primary_keys)
        else:
            assert False, f"unrecognized bucketization method: {bucket_method}"

        binned_data.update(temp_data)
        for K in equivalent_keys[PK]:
            optimal_buckets[K] = optimal_bucket
            temp_table_name = K.split(".")[0]
            if temp_table_name not in bin_size:
                bin_size[temp_table_name] = dict()
            bin_size[temp_table_name][K] = len(optimal_bucket.bins)
            all_bin_modes[K] = np.asarray(optimal_bucket.buckets[K].bin_modes)

    table_buckets = generate_table_buckets(data, key_attrs, bin_size, all_bin_modes, optimal_buckets)

    for K in binned_data:
        temp_table_name = K.split(".")[0]
        temp = data[temp_table_name][K].values
        temp[temp >= 0] = binned_data[K]

    if save_bucket_bins:
        with open(model_folder + f"/buckets.pkl") as f:
            pickle.dump(optimal_buckets, f, pickle.HIGHEST_PROTOCOL)

    return data, null_values, key_attrs, table_buckets, equivalent_keys, schema, bin_size

In [ ]:
data_path = "/home/ubuntu/End-to-End-CardEst-Benchmark/datasets/stats_simplified/{}.csv"
model_folder = "/home/ubuntu/data_CE/saved_models"
data, null_values, key_attrs, table_buckets, equivalent_keys, schema, bin_size = process_stats_data(data_path,
                                                                       model_folder, 200, "sub_optimal", False)

In [ ]:
df_rows = pd.read_csv(data_path.format("badges"))
df_rows.head(5)

In [ ]:
table = "votes"
bucket = table_buckets[table]
for attr in bucket.id_attributes:
    print(attr)
    print(np.sum(bucket.oned_bin_modes[attr]), bucket.oned_bin_modes[attr].shape)
    if len(bucket.twod_bin_modes[attr]) != 0:
        print(np.sum(bucket.twod_bin_modes[attr]), bucket.twod_bin_modes[attr].shape)

In [ ]:
bucket.twod_bin_modes[attr]

In [ ]:
table = "postHistory"
bucket = table_buckets[table]
for attr in bucket.id_attributes:
    print(attr)
    print(np.sum(bucket.oned_bin_modes[attr]), bucket.oned_bin_modes[attr].shape)
    if len(bucket.twod_bin_modes[attr]) != 0:
        print(np.sum(bucket.twod_bin_modes[attr]), bucket.twod_bin_modes[attr].shape)

In [ ]:
print(bucket.oned_bin_modes[bucket.id_attributes[0]])
print(np.sum(bucket.twod_bin_modes[bucket.id_attributes[0]], axis=1))
print(bucket.oned_bin_modes[bucket.id_attributes[1]])
print(np.sum(bucket.twod_bin_modes[bucket.id_attributes[1]], axis=0))

In [ ]:
a = np.arange(1000)


In [ ]:
_, bins = np.histogram(a, bins=30)

In [ ]:
a[np.where((a<30) & (a>15))]